##### Ensure the person has Installed Librosa, Tensorflow and Keras
- If a LOT of "requirement already satisfied is found" then the person most likely has everything installed and good to go

In [1]:
# ensure everyone has librosa, tensorflow, and keras
import sys
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras

In [2]:
# feature extraction and data preprocessing
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

# Acquire and Prepare Dataset
- Read in audio features from the GTZAN dataset. (download from: https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification)
-- 100 songs per genre, 10 genres

In [3]:
data = pd.read_csv('DATA/features_30_sec.csv')
data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [4]:
data.shape

(1000, 60)

In [5]:
# Dropping unneccesary or not yet known from class columns
data = data.drop(['filename', 'length', 'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var','tempo'],axis=1)
data.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


- Encode the Labels

In [6]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

- Scaling the Feature columns

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
print(X.shape)

(1000, 52)


# Setup Training vs Testing Data
- Dividing data into training and Testing set

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
len(y_train)

800

In [10]:
len(y_test)

200

In [11]:
X_train[10]

array([ 0.89705486, -1.37338524,  0.16444967, -0.66959416,  0.42225199,
       -0.76570287, -0.1178408 , -1.0686537 ,  0.32136097, -1.01605753,
        1.52922181, -0.14416283,  0.61857086, -0.97656502, -0.17246859,
       -1.13691108, -0.83862168, -1.05130658,  2.36365155, -1.42137337,
       -2.15868501, -1.05716648,  1.76934493, -1.23806224, -1.52601884,
       -1.13279942,  1.43351702, -0.81747808, -1.01048403, -1.02459143,
        0.45562146, -0.67597534, -1.08199931, -0.5414882 ,  0.54628468,
       -0.56099889, -0.60015038, -0.10144141,  1.30037016,  0.23148609,
       -1.32922932,  0.26904841, -1.00860866, -0.31173874, -0.2772415 ,
       -0.88515255, -0.80064826, -0.74251444, -1.60637894,  0.17240584,
       -1.53543852, -0.75676163])

# Setup Network Arhitecture
- We setup a feed forward deep learning neural network.
- It has 5 densly connected layers. The 1st 4 layers have relu activation, and the final layer is softmax.

In [12]:
from keras import models
from keras import layers

model = models.Sequential()
#    Dense( Layers "Neurons", Activation Type, input)
# reduce neurons for consolidation (Hidden Layers)
# at 10 neurons, softmax is our output layer
model.add(layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(512, activation='relu'))#, input_shape=(X_train.shape[1],)))

#model.add(layers.Dense(256, activation='relu'))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

# Define Network Loss and Optimization

In [13]:
model.compile(optimizer='RMSprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the network
- Optimization Notes:
- BEST optimizations BEFORE dropping tempo, harm, and perc
 - RMSprop: ~83% to 84%, epochs=800, batch=128, 9-1 train test split, exclude layer 128
 - adam/adamax: 77%, epochs=1000, batch=64, 9-1 train test split
 - sgd:  76%, same as above
 - ftrl: 70%, same as above
   - anything below 9:1 train split results in below 80 or even 70% acc

- BEST optimizations AFTER dropping tempo, harm, and perc
 - adam: 75%, epoch = 1000, batch=128, 0.1 test ratio, exclude layer 256
 - RMSprop: 79.5%, epoch 1k, batch 128, 0.2 test size, exclude layer 256

In [14]:
history = model.fit(X_train,
                    y_train,
                    epochs=1000,
                    batch_size=128)

Epoch 1/1000
7/7 [==============================] - 1s 8ms/step - loss: 1.8113 - accuracy: 0.3600
Epoch 2/1000
7/7 [==============================] - 0s 8ms/step - loss: 1.2047 - accuracy: 0.5775
Epoch 3/1000
7/7 [==============================] - 0s 9ms/step - loss: 1.0586 - accuracy: 0.6212
Epoch 4/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.8339 - accuracy: 0.7100
Epoch 5/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.6696 - accuracy: 0.7850
Epoch 6/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6738 - accuracy: 0.7588
Epoch 7/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.5281 - accuracy: 0.8225
Epoch 8/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.5101 - accuracy: 0.8150
Epoch 9/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.4525 - accuracy: 0.8550
Epoch 10/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.3965 - accuracy: 0.8725
Epoch 11/1000
7/7 [

7/7 [==============================] - 0s 8ms/step - loss: 0.0058 - accuracy: 0.9975
Epoch 84/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0035 - accuracy: 0.9987
Epoch 85/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0029 - accuracy: 0.9987
Epoch 86/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0028 - accuracy: 0.9975
Epoch 87/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0052 - accuracy: 0.9987
Epoch 88/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0033 - accuracy: 0.9975
Epoch 89/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0027 - accuracy: 0.9975
Epoch 90/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0028 - accuracy: 0.9987
Epoch 91/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0029 - accuracy: 0.9975
Epoch 92/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0030 - accuracy: 0.9975
Epoch 93/1000
7/7 [=====

7/7 [==============================] - 0s 8ms/step - loss: 0.0023 - accuracy: 0.9987
Epoch 166/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0617 - accuracy: 0.9862
Epoch 167/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0026 - accuracy: 0.9987
Epoch 168/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0021 - accuracy: 0.9987
Epoch 169/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 0.9975
Epoch 170/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 0.9975
Epoch 171/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0021 - accuracy: 0.9975
Epoch 172/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0022 - accuracy: 0.9987
Epoch 173/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0022 - accuracy: 0.9987
Epoch 174/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0022 - accuracy: 0.9987
Epoch 175/1000


7/7 [==============================] - 0s 10ms/step - loss: 0.1980 - accuracy: 0.9650
Epoch 248/1000
7/7 [==============================] - 0s 10ms/step - loss: 0.0133 - accuracy: 0.9962
Epoch 249/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0026 - accuracy: 0.9987
Epoch 250/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0021 - accuracy: 0.9975
Epoch 251/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0020 - accuracy: 0.9987
Epoch 252/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 253/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0020 - accuracy: 0.9987
Epoch 254/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0020 - accuracy: 0.9975
Epoch 255/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 0.9987
Epoch 256/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 0.9987
Epoch 257/100

7/7 [==============================] - 0s 9ms/step - loss: 0.0053 - accuracy: 0.9987
Epoch 330/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0577 - accuracy: 0.9850
Epoch 331/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0048 - accuracy: 0.9975
Epoch 332/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 333/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 0.9975
Epoch 334/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0021 - accuracy: 0.9987
Epoch 335/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0021 - accuracy: 0.9975
Epoch 336/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0017 - accuracy: 0.9987
Epoch 337/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0023 - accuracy: 0.9975
Epoch 338/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0022 - accuracy: 0.9987
Epoch 339/1000


...

7/7 [==============================] - 0s 9ms/step - loss: 0.0021 - accuracy: 0.9975
Epoch 658/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0019 - accuracy: 0.9987
Epoch 659/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - accuracy: 0.9987
Epoch 660/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 661/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0017 - accuracy: 0.9987
Epoch 662/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0024 - accuracy: 0.9987
Epoch 663/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0019 - accuracy: 0.9987
Epoch 664/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 0.9975
Epoch 665/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 666/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.3691 - accuracy: 0.9613
Epoch 667/1000


7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 740/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0019 - accuracy: 0.9975
Epoch 741/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 742/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0019 - accuracy: 0.9975
Epoch 743/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0019 - accuracy: 0.9975
Epoch 744/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 745/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 746/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0019 - accuracy: 0.9987
Epoch 747/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.3443 - accuracy: 0.9650
Epoch 748/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0100 - accuracy: 0.9950
Epoch 749/1000


7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 822/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 823/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 824/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 825/1000
7/7 [==============================] - 0s 10ms/step - loss: 0.0017 - accuracy: 0.9987
Epoch 826/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0020 - accuracy: 0.9975
Epoch 827/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - accuracy: 0.9975
Epoch 828/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 829/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0017 - accuracy: 0.9987
Epoch 830/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0017 - accuracy: 0.9987
Epoch 831/1000

7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 903/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0063 - accuracy: 0.9975
Epoch 904/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 905/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 906/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 907/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 908/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 909/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 910/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 911/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 912/1000


7/7 [==============================] - 0s 9ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 985/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9975
Epoch 986/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0025 - accuracy: 0.9987
Epoch 987/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 988/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0029 - accuracy: 0.9975
Epoch 989/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0021 - accuracy: 0.9975
Epoch 990/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 0.9975
Epoch 991/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.0019 - accuracy: 0.9975
Epoch 992/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 993/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 994/1000


# Evaluate the model with our testing data

In [15]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 2ms/step - loss: 5.6299 - accuracy: 0.7200


In [16]:
print('test_acc: ',test_acc)

test_acc:  0.7200000286102295


# Model Inference 
- Predictions from the network for my playlist of songs

In [17]:
# get playlist data from csv
musicData = pd.read_csv('DATA/manipAudio.csv')
musicData.head()

,filename,chroma_stft_mean,chroma_stft_var,rmse_mean,rmse_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,JimiHendrix.PurpleHaze.wav,0.455856,0.082303,0.188019,0.003082,2362.116655,398426.151640,2108.223718,96814.201568,4503.639434,...,32.462551,-15.391335,36.343109,-0.342875,46.522861,3.693954,57.158340,-0.031445,54.754475,NaN
1,RobertJohnson.HellhoundOnMyTrail.wav,0.272026,0.090724,0.061344,0.000860,1495.770517,128649.989989,1026.712815,10848.554246,2666.708875,...,99.860817,-15.647900,122.161064,-10.185360,177.957520,4.993184,242.663696,1.418939,211.229248,NaN
2,Ozzy.CrazyTrain.wav,0.368314,0.079328,0.213740,0.002873,2280.488043,110004.683801,2331.200451,29202.862549,4793.562681,...,36.335785,-1.631311,43.356983,2.425800,29.732515,-5.956182,21.572521,-1.423873,28.247942,NaN
3,Above&Beyond.Sun&Moon.wav,0.457347,0.092967,0.169338,0.002347,2522.054084,823251.777594,2838.415560,330720.426357,6001.138712,...,56.962280,-1.739794,62.940365,2.775500,47.585907,-2.514384,59.070450,1.939490,51.466339,NaN
4,BoneyM.DaddyCool.wav,0.373412,0.078024,0.129959,0.000611,2560.867238,229624.776926,2530.922549,103294.402065,5378.792475,...,39.186672,-8.458516,27.143917,-0.405075,29.694092,-5.304452,34.051109,5.641595,45.173008,NaN


In [18]:
# strip song information from the filename column
def getSongInfo(string):
    return string.split(".")[1] + " by " + string.split(".")[0]
    musicData.iloc[0][0].split(".")[1]

In [19]:
# get all song names before dropping that data
songTitles = ['']*5
genreList = ['rock','blues','metal','EDM','disco'] # this is in order
defGenres    = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock',]
for i in range(5):
    songTitles[i] = getSongInfo(musicData.iloc[i][0])
    
songTitles[2]

'CrazyTrain by Ozzy'

In [20]:
# Drop filenames to start predictions
musicData = musicData.drop(['filename'], axis=1)
myX = scaler.fit_transform(np.array(musicData.iloc[:,:-1], dtype = float))
print(myX.shape)
musicData.head()

(5, 52)


,chroma_stft_mean,chroma_stft_var,rmse_mean,rmse_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.455856,0.082303,0.188019,0.003082,2362.116655,398426.151640,2108.223718,96814.201568,4503.639434,1.186836e+06,...,32.462551,-15.391335,36.343109,-0.342875,46.522861,3.693954,57.158340,-0.031445,54.754475,NaN
1,0.272026,0.090724,0.061344,0.000860,1495.770517,128649.989989,1026.712815,10848.554246,2666.708875,5.272243e+04,...,99.860817,-15.647900,122.161064,-10.185360,177.957520,4.993184,242.663696,1.418939,211.229248,NaN
2,0.368314,0.079328,0.213740,0.002873,2280.488043,110004.683801,2331.200451,29202.862549,4793.562681,3.362283e+05,...,36.335785,-1.631311,43.356983,2.425800,29.732515,-5.956182,21.572521,-1.423873,28.247942,NaN
3,0.457347,0.092967,0.169338,0.002347,2522.054084,823251.777594,2838.415560,330720.426357,6001.138712,4.277782e+06,...,56.962280,-1.739794,62.940365,2.775500,47.585907,-2.514384,59.070450,1.939490,51.466339,NaN
4,0.373412,0.078024,0.129959,0.000611,2560.867238,229624.776926,2530.922549,103294.402065,5378.792475,1.466999e+06,...,39.186672,-8.458516,27.143917,-0.405075,29.694092,-5.304452,34.051109,5.641595,45.173008,NaN


In [21]:
# predict the genres of each song
playlist = myX
predictions = model.predict(playlist)

###### Prediction Debug Output

In [22]:
print(playlist.shape)
print(predictions.shape)
print("Purple Haze Stats...")
print(predictions[0], np.sum(predictions[0]), np.argmax(predictions[0]))

(5, 52)
(5, 10)
Purple Haze Stats...
[8.39628419e-28 8.36274601e-22 1.37211281e-24 9.99995828e-01
 5.50368724e-13 5.89274369e-18 1.39258700e-11 1.43691855e-11
 5.53494601e-08 4.11733208e-06] 1.0 3


## Output and Output Formatting

In [23]:
# populate song and genre dictionary
songToGenre = {}
index = 0
for s in predictions:
    gguess = np.argmax(s)
    gmask = np.zeros(10, dtype=int) #size of playlist
    gmask[gguess] = 1
    gsecond = np.argmax(np.ma.masked_array(s, mask=gmask))
    songToGenre.setdefault(songTitles[index], [(gguess, s[gguess]), (gsecond, s[gsecond])])
    index = index + 1

In [24]:
print(songToGenre)

{'PurpleHaze by JimiHendrix': [(3, 0.9999958), (9, 4.117332e-06)], 'HellhoundOnMyTrail by RobertJohnson': [(0, 0.9999788), (5, 2.1192809e-05)], 'CrazyTrain by Ozzy': [(4, 0.9990181), (6, 0.0009819107)], 'Sun&Moon by Above&Beyond': [(8, 0.7400555), (4, 0.2599445)], 'DaddyCool by BoneyM': [(3, 1.0), (6, 5.8806776e-10)]}


In [25]:
# output formatting
index = 0
for k in songToGenre.keys():
    gOne = songToGenre[k][0][0] #.first
    gTwo = songToGenre[k][1][0] #.first
    pOne = songToGenre[k][0][1] #.second
    pTwo = songToGenre[k][1][1] #.second
    print("%s is expected to be %s or %s. These have confidence scores of %.2f%% and %.2f%%." % 
          (k, defGenres[gOne], defGenres[gTwo], (pOne * 100), (pTwo * 100)))
    print("It is actually %s.\n" % genreList[index])
    index = index + 1

PurpleHaze by JimiHendrix is expected to be disco or rock. These have confidence scores of 100.00% and 0.00%.
It is actually rock.

HellhoundOnMyTrail by RobertJohnson is expected to be blues or jazz. These have confidence scores of 100.00% and 0.00%.
It is actually blues.

CrazyTrain by Ozzy is expected to be hiphop or metal. These have confidence scores of 99.90% and 0.10%.
It is actually metal.

Sun&Moon by Above&Beyond is expected to be reggae or hiphop. These have confidence scores of 74.01% and 25.99%.
It is actually EDM.

DaddyCool by BoneyM is expected to be disco or metal. These have confidence scores of 100.00% and 0.00%.
It is actually disco.



##### My Results from taking first 30 seconds of a song
PurpleHaze by JimiHendrix is expected to be metal or rock. These have confidence scores of 92.30 and 7.70.
It is actually rock.

HellhoundOnMyTrail by RobertJohnson is expected to be country or classical. These have confidence scores of 100.00 and 0.00.
It is actually blues.

CrazyTrain by Ozzy is expected to be metal or hiphop. These have confidence scores of 98.80 and 1.20.
It is actually metal.

Sun&Moon by Above&Beyond is expected to be reggae or hiphop. These have confidence scores of 100.00 and 0.00.
It is actually EDM.

DaddyCool by BoneyM is expected to be disco or classical. These have confidence scores of 100.00 and 0.00.
It is actually disco.

##### My Results taking the 30 second sound clip from after the beginning of every song

PurpleHaze by JimiHendrix is expected to be rock or hiphop. These have confidence scores of 89.25% and 10.75%.
It is actually rock.

HellhoundOnMyTrail by RobertJohnson is expected to be jazz or blues. These have confidence scores of 88.58% and 11.42%.
It is actually blues.

CrazyTrain by Ozzy is expected to be blues or metal. These have confidence scores of 74.30% and 25.70%.
It is actually metal.

Sun&Moon by Above&Beyond is expected to be reggae or hiphop. These have confidence scores of 100.00% and 0.00%.
It is actually EDM.

DaddyCool by BoneyM is expected to be disco or country. These have confidence scores of 99.98% and 0.02%.
It is actually disco.

In [26]:
# End Assignment